In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets('./mnist/data', one_hot=True)

Instructions for updating:
Use the retry module or similar alternatives.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz
Instru

In [ ]:
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32, [None, 10])

### 미니 배치

이미지를 하나씩 학습시키는 것보다 여러개를 한꺼번에 학습시키는 것이 효과가 좋지만, 컴퓨팅 성능의 제약 때문에 적당한 크기로 잘라서 학습시킨다. 이것을 미니배치라고 한다. X, Y 텐서의 첫번째 차원이 None으로 되어있는데, 이 자리에는 이미지 개수를 지정하는 값인 배치 크기를 지정하게 된다. 원하는 배치 크기로 명시할 수 있지만 개수를 계속 변경하면서 실험해보려면 None으로 하는것이 유리하다.

In [3]:
# 신경망 개수 
# 784(입력, 특징 개수, 28 * 28) -> 256 (첫 번째 은닉층 뉴런 개수) -> 256 (두 번째 은닉층 뉴런 개수) -> 10 (결과값 0,9 분류 개수)

W1 = tf.Variable(tf.random_normal([784, 256], stddev=0.01))
b1 = tf.Variable(tf.zeros([256]))
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))

W2 = tf.Variable(tf.random_normal([256, 256], stddev=0.01))
b2 = tf.Variable(tf.zeros([256]))
L2 = tf.nn.relu(tf.add(tf.matmul(L1, W2), b2))

W3 = tf.Variable(tf.random_normal([256, 10], stddev=0.01))
b3 = tf.Variable(tf.zeros([10]))
model = tf.add(tf.matmul(L2, W3), b3)


- `tf.random_normal`: 정규 분포를 가지는 임의의 값. stddev는 표준편차를 지정하는 파라미터.  

In [4]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(0.001).minimize(cost)

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

### training data vs test data
`training data` 학습데이터는 학습을 시킬 때 사용하고, `test data` 테스트 데이터는 학습을 테스트 하기 위해서 사용한다. 왜냐하면 학습 데이터는 예측을 매우 잘하지만, 실제 데이터(학습 데이터에 포합되지 않은 데이터)는 정확도가 떨어지는 상태를 피하기 위해서이다. 이런 상태를 `Overfitting` 과적합이라고 한다.

MNIST 데이터 셋의 경우, 학습데이터 6만개와 테스트 데이터 1만개로 구성되어 있다.

In [6]:
batch_size = 100
total_batch = int(mnist.train.num_examples / batch_size)

print('batch size : %s' % batch_size)
print('total batch size : %s' % total_batch)

batch size : 100
total batch size : 550


MNIST는 데이터가 수만개이므로 미니배치를 이용하여 학습한다. 미니배치를 100으로 지정한다. 결과 미니배치의 개수는 총 550개이다.

In [11]:
for epoch in range(15):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([optimizer, cost], feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val
        
    print('Epoch: %04d' % (epoch + 1), 'Avg. cost = {:.3f}'.format(total_cost/total_batch))

print('Complete!')

Epoch: 0001 Avg. cost = 0.005
Epoch: 0002 Avg. cost = 0.005
Epoch: 0003 Avg. cost = 0.007
Epoch: 0004 Avg. cost = 0.005
Epoch: 0005 Avg. cost = 0.005
Epoch: 0006 Avg. cost = 0.005
Epoch: 0007 Avg. cost = 0.004
Epoch: 0008 Avg. cost = 0.004
Epoch: 0009 Avg. cost = 0.009
Epoch: 0010 Avg. cost = 0.002
Epoch: 0011 Avg. cost = 0.003
Epoch: 0012 Avg. cost = 0.007
Epoch: 0013 Avg. cost = 0.006
Epoch: 0014 Avg. cost = 0.002
Epoch: 0015 Avg. cost = 0.004
Complete!


### Epoch
학습데이터 전체를 한바퀴 돌아서 학습하는 것을 `epoch` 에폭 이라고 한다.

In [27]:
is_correct = tf.equal(tf.argmax(model, axis=1), tf.argmax(Y, axis=1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('Accuracy : %s' % sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Accuracy : 0.9768


예측한 결괏값은 원핫 인코딩 형식이며 각 인덱스에 해당하는 값은 얼마나 해당 인덱스와 관련이 높은가를 나타낸다. 이것은 손실값을 `softmax_cross_entropy_with_logits`를 이용했기 때문이다.

- `tf.argmax(model, axis=1)`: model 두번째 차원(1번 인덱스의 차원)의 값중 최댓값의 인덱스를 뽑아내는 함수. 그래서 결과가 1차원 배열이 된다. 

In [28]:
print('Model : %s' % sess.run(model, feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

Model : [[-13.400582  -18.154526  -13.321923  ...  22.286652  -22.17606
    3.9002728]
 [-12.15994    -2.7420294  37.294502  ... -47.60979   -33.038536
  -44.613136 ]
 [-27.639322   14.161953   -5.440674  ...  -4.740999    1.9001095
  -19.939383 ]
 ...
 [-38.794693  -30.791235  -27.184042  ... -10.2394705 -18.221363
   -3.6161861]
 [-32.935997  -32.811287  -44.752804  ... -22.837934   -4.5150557
  -42.987896 ]
 [ -9.561734  -37.685596  -20.247814  ... -49.692673  -31.70034
  -51.061157 ]]


In [29]:
print('tf.argmax(model, 1) : %s' % sess.run(tf.argmax(model, 1), feed_dict={X: mnist.test.images, Y: mnist.test.labels}))

tf.argmax(model, 1) : [7 2 1 ... 4 5 6]
